# I. Thư viện

In [3]:
import cv2
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization
import torch
from torchvision import transforms
import numpy as np
from PIL import Image
import time

# II. Chuẩn bị

Gọi GPU

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [5]:
data_path = './data/annotations'

Tải dữ liệu

In [6]:
embeddings= torch.load(data_path + '/facelist.pth')
names = np.load(data_path + '/username.npy')

Lấy camera

In [7]:
video_cap = cv2.VideoCapture(0)
video_cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
video_cap.set(cv2.CAP_PROP_FRAME_HEIGHT,480)

True

Kích thước đầu vào lấy từ webcam

In [8]:
frame_size =  (640, 480)

# III Chuẩn hóa

Model MTCNN để tách khuôn mặt

In [9]:
mtcnn = MTCNN(thresholds= [0.7, 0.7, 0.8] ,keep_all=True, device = device)

Các hàm chuẩn hóa

In [10]:
def standardization(img_tensor_tmp):
    normalize = (img_tensor_tmp -127.5)/128
    return normalize

In [11]:
def trans(img):
    transform = transforms.Compose([
        transforms.ToTensor(),
        fixed_image_standardization
    ])
    return transform(img)

# IV. Main

Khởi tạo model InceptionResnetV1

In [12]:
model = InceptionResnetV1(
    classify=False,
    pretrained="casia-webface"
).to(device)

model.eval()

InceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_4a): 

### Funtion

extract_face: Trích xuất khuôn mặt từ các bounding box
Thông số:
- margin: Tương đương với margin khi capture_face ( để là 20)


In [13]:
def extract_face(box, img, margin=20):
    face_size = 160
    img_size = frame_size
    margin = [
        margin * (box[2] - box[0]) / (face_size - margin),
        margin * (box[3] - box[1]) / (face_size - margin),
    ] #tạo margin bao quanh box cũ
    box = [
        int(max(box[0] - margin[0] / 2, 0)),
        int(max(box[1] - margin[1] / 2, 0)),
        int(min(box[2] + margin[0] / 2, img_size[0])),
        int(min(box[3] + margin[1] / 2, img_size[1])),
    ]
    img = img[box[1]:box[3], box[0]:box[2]]
    face = cv2.resize(img,(face_size, face_size), interpolation=cv2.INTER_AREA)
    face = Image.fromarray(face)
    return face

Kết xuất embedding cho từng ảnh mặt - thứ đã được extract từ hàm extract_face() ở trên

In [14]:
def inference(model, face, local_embeds, threshold = 3):
    #local: [n,512] voi n la so nguoi trong faceslist
    embeds = []
    # print(trans(face).unsqueeze(0).shape)
    embeds.append(model(trans(face).to(device).unsqueeze(0)))
    detect_embeds = torch.cat(embeds) #[1,512]
    # print(detect_embeds.shape)
    #[1,512,1]                                      [1,512,n]
    norm_diff = detect_embeds.unsqueeze(-1) - torch.transpose(local_embeds, 0, 1).unsqueeze(0)
    # print(norm_diff)
    norm_score = torch.sum(torch.pow(norm_diff, 2), dim=1) #(1,n), moi cot la tong khoang cach euclide so vs embed moi

    min_dist, embed_idx = torch.min(norm_score, dim = 1)
    print(min_dist*power, names[embed_idx])
    # print(min_dist.shape)
    if min_dist*power > threshold:
        return -1, -1
    else:
        return embed_idx, min_dist.double()

main

In [15]:
while video_cap.isOpened():
    isSuccess, frame = video_cap.read()
    if isSuccess:
        boxes, _ = mtcnn.detect(frame)
        if boxes is not None:
            for box in boxes:
                bbox = list(map(int,box.tolist()))
                face = extract_face(bbox, frame)
                idx, score = inference(model, face, embeddings)
                if idx != -1:
                    frame = cv2.rectangle(frame, (bbox[0],bbox[1]), (bbox[2],bbox[3]), (0,0,255), 6)
                    score = torch.Tensor.cpu(score[0]).detach().numpy()*power
                    frame = cv2.putText(frame, names[idx] + '_{:.2f}'.format(score), (bbox[0],bbox[1]), cv2.FONT_HERSHEY_DUPLEX, 2, (0,255,0), 2, cv2.LINE_8)
                else:
                    frame = cv2.rectangle(frame, (bbox[0],bbox[1]), (bbox[2],bbox[3]), (0,0,255), 6)
                    frame = cv2.putText(frame,'Unknown', (bbox[0],bbox[1]), cv2.FONT_HERSHEY_DUPLEX, 2, (0,255,0), 2, cv2.LINE_8)

        new_frame_time = time.time()
        fps = 1/(new_frame_time-prev_frame_time)
        prev_frame_time = new_frame_time
        fps = str(int(fps))
        cv2.putText(frame, fps, (7, 70), cv2.FONT_HERSHEY_DUPLEX, 3, (100, 255, 0), 3, cv2.LINE_AA)

    cv2.imshow('Face Recognition', frame)
    if cv2.waitKey(1)&0xFF == 27:
        break

tensor([0.2446], device='cuda:0', grad_fn=<MulBackward0>) Cong
tensor([0.0524], device='cuda:0', grad_fn=<MulBackward0>) Cong
tensor([0.2927], device='cuda:0', grad_fn=<MulBackward0>) Cong
tensor([0.2442], device='cuda:0', grad_fn=<MulBackward0>) Cong
tensor([0.2442], device='cuda:0', grad_fn=<MulBackward0>) Cong
tensor([0.1429], device='cuda:0', grad_fn=<MulBackward0>) Cong
tensor([0.1821], device='cuda:0', grad_fn=<MulBackward0>) Cong
tensor([0.2088], device='cuda:0', grad_fn=<MulBackward0>) Cong
tensor([0.2584], device='cuda:0', grad_fn=<MulBackward0>) Cong
tensor([0.2664], device='cuda:0', grad_fn=<MulBackward0>) Cong
tensor([0.2027], device='cuda:0', grad_fn=<MulBackward0>) Cong
tensor([0.2051], device='cuda:0', grad_fn=<MulBackward0>) Cong
tensor([1.0184], device='cuda:0', grad_fn=<MulBackward0>) Cong
tensor([0.3123], device='cuda:0', grad_fn=<MulBackward0>) Cong
tensor([0.3194], device='cuda:0', grad_fn=<MulBackward0>) Cong
tensor([0.3416], device='cuda:0', grad_fn=<MulBackward0

In [16]:
video_cap.release()
cv2.destroyAllWindows()